In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import lxml
import pandas as pd
import numpy as np 
import re

# PART 1 -- Trying to generate an extract all the links available on the main page 

In [3]:
url = 'https://results.eci.gov.in/'
response = requests.get(url)
html_main_file = response.text

bs4_obj = BeautifulSoup(html_main_file,'lxml')

In [4]:
url_link = []
link = bs4_obj.find_all('a')
for links in link:
    print(links['href'])
    url_link.append(links['href'])

index.htm
https://results.eci.gov.in/PcResultGenJune2024/index.htm
https://results.eci.gov.in/AcResultGenJune2024/index.htm
https://results.eci.gov.in/AcResultGenJune2024/index.htm
https://results.eci.gov.in/AcResultByeJune2024/
https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
https://play.google.com/store/apps/details?id=com.eci.citizen
https://apps.apple.com/in/app/voter-helpline/id1456535004


In [5]:
url_link = url_link[:-2]
print(url_link)

['index.htm', 'https://results.eci.gov.in/PcResultGenJune2024/index.htm', 'https://results.eci.gov.in/AcResultGenJune2024/index.htm', 'https://results.eci.gov.in/AcResultGenJune2024/index.htm', 'https://results.eci.gov.in/AcResultByeJune2024/', 'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm', 'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm']


In [6]:
url_link = url_link[1:]
print(url_link)

['https://results.eci.gov.in/PcResultGenJune2024/index.htm', 'https://results.eci.gov.in/AcResultGenJune2024/index.htm', 'https://results.eci.gov.in/AcResultGenJune2024/index.htm', 'https://results.eci.gov.in/AcResultByeJune2024/', 'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm', 'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm']


# PART 2 - Extracting data from the following links of the url_link

Part 2.1 - Now Let's extarct Data from the First Link which describes the result of the parliamentary contituencies


1&2 -> Here we have two major insights one being how seats each party have along with percentage and the total no. of votes given along with the percentage of vote recieved by the each party.

3 -> We also have the insight on how many seats are there

In [7]:
response_link_1 = requests.get(url_link[0])
response_link_1 = response_link_1.text
rs1_obj = BeautifulSoup(response_link_1,'html.parser')

In [8]:
table = rs1_obj.find_all('table')
table

[<table class="table">
 <thead>
 <tr>
 <th style="text-align: left;">Party</th>
 <th style="text-align:right">Won</th>
 <th style="text-align:right">Leading</th>
 <th style="text-align:right">Total</th>
 </tr>
 </thead>
 <tbody>
 <tr class="tr">
 <td style="text-align:left">Bharatiya Janata Party - BJP</td>
 <td style="text-align:right">
 <a href="partywisewinresultState-369.htm">240</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                                                     </td>
 <td style="text-align:right">240</td>
 </tr>
 <tr class="tr">
 <td style="text-align:left">Indian National Congress - INC</td>
 <td style="text-align:right">
 <a href="partywisewinresultState-742.htm">99</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                   

In [9]:
heading_table = []
for tr in rs1_obj.find('tr').find_all('th'):
    print(tr.text.strip())
    heading_table.append(tr.text.strip())

Party
Won
Leading
Total


In [10]:
table_data = []
for tr in rs1_obj.find_all('tr')[1:]:
    for td in tr.find_all('td'):
        table_data.append(td.text.strip())

In [11]:
table_data[4]

'Indian National Congress - INC'

In [12]:
table_data = np.array(table_data)
table_data = table_data.reshape(42,4)
df1 = pd.DataFrame(table_data,columns=heading_table)

In [13]:
df1.head()

,Party,Won,Leading,Total
0,Bharatiya Janata Party - BJP,240,0,240
1,Indian National Congress - INC,99,0,99
2,Samajwadi Party - SP,37,0,37
3,All India Trinamool Congress - AITC,29,0,29
4,Dravida Munnetra Kazhagam - DMK,22,0,22


In [14]:
df1.tail()

,Party,Won,Leading,Total
37,Aazad Samaj Party (Kanshi Ram) - ASPKR,1,0,1
38,Apna Dal (Soneylal) - ADAL,1,0,1
39,AJSU Party - AJSUP,1,0,1
40,All India Majlis-E-Ittehadul Muslimeen - AIMIM,1,0,1
41,Independent - IND,7,0,7


In [15]:
df1['Total'] = df1['Total'].astype(int)

In [16]:
total_parlia_seats = df1['Total'].sum()

In [17]:
df1['Seat_Share'] = (df1['Total']/df1['Total'].sum())*100

In [18]:
df1.head()

,Party,Won,Leading,Total,Seat_Share
0,Bharatiya Janata Party - BJP,240,0,240,44.198895
1,Indian National Congress - INC,99,0,99,18.232044
2,Samajwadi Party - SP,37,0,37,6.813996
3,All India Trinamool Congress - AITC,29,0,29,5.340700
4,Dravida Munnetra Kazhagam - DMK,22,0,22,4.051565


In [19]:
# this particlar script has a the information about the vote share for each 
script_list = rs1_obj.find_all('script')[-1]
script_list

<script type="text/javascript">

    // Doughnut Charts Starts Here
    var xValues = ['BJP','INC','SP','AITC','DMK','TDP','JD(U)','SHSUBT','NCPSP','SHS','Others'];
    var yValues = [240,99,37,29,22,16,12,9,8,7,64];
    var barColors = ['#ff944d','#19AAED','#ff0000','#aebedf','#05F86E','#39AC57','#204795','#4FB825','#D781F9','#D2691E','#b3b3b3'];

    /* var xValues = ["AITC", "BJP", "HSPDP", "INC", "JD", "NOTA", "NPEP", "PDF", "UDP", "Other"];
    var yValues = [55, 49, 44, 24, 15, 55, 49, 44, 24, 15];
    var barColors = [
      "#ff9f40",
      "#ffcd56",
      "#4bc0c0",
      "#36a2eb",
      "#9966ff",
      "#aebedf",
      "#ff6384",
      "#fb9a74",
      "#b5d36e",
      "#e5aad4"
    ]; */

    new Chart("donutChart", {
      type: "doughnut",
      data: {
        labels: xValues,
        datasets: [{
          backgroundColor: barColors,
          data: yValues
        }]
      },
      options: {
        rotation: -90,
        circumference: 180,
        responsive: true

Since there is some difficulty accessing the yValue with the no.of vote we will just copy them 


In [20]:
Vote_party = ['AAAP{1.11%}','ADAL{0.13%}','ADMK{1.39%}','AGP{0.20%}','AIFB{0.04%}','AIMIM{0.22%}','AITC{4.37%}','AIUDF{0.10%}','AJSUP{0.07%}','BHRS{0.57%}','BJD{1.46%}','BJP{36.56%}','BOPF{0.12%}','BSP{2.04%}','CPI{0.49%}','CPI(M){1.76%}','CPI(ML)(L){0.27%}','DMDK{0.17%}','DMK{1.82%}','INC{21.19%}','INLD{0.04%}','IUML{0.27%}','JD(S){0.34%}','JD(U){1.25%}','JKN{0.18%}','JKNPPB{0.00%}','JKPDP{0.07%}','JMM{0.41%}','JNJP{0.02%}','KEC(M){0.04%}','LJPRV{0.44%}','MNF{0.02%}','NCP{0.32%}','NCPSP{0.92%}','NDPP{0.05%}','NOTA{0.99%}','NPEP{0.06%}','NPF{0.05%}','RJD{1.57%}','RLTP{0.09%}','RVLTGONP{0.01%}','SAD{0.28%}','SDF{0.01%}','SHS{1.15%}','SHSUBT{1.48%}','SKM{0.03%}','SP{4.58%}','TDP{1.98%}','UDP{0.01%}','UPPL{0.08%}','VOTPP{0.09%}','YSRCP{2.06%}','ZPM{0.03%}','Others{7.03%}']
Vote_count= [7147800,808245,8952587,1298707,289941,1400215,28213393,625954,458677,3657237,9413379,235973935,777570,13153818,3157184,11342553,1736771,1128616,11754710,136759064,226975,1716186,2173701,8039663,1147041,23268,435980,2652955,113827,277365,2810250,140264,2059179,5921162,350967,6372220,417930,299536,10107402,596955,64578,1814318,77171,7401447,9567779,164396,29549381,12775270,44563,488995,571078,13316039,208552,45386696]

In [21]:
Party_name = []
for i in Vote_party:
    x,y = i.split('{')
    Party_name.append(x)


In [22]:
Party_name[1:3]

['ADAL', 'ADMK']

In [23]:
Vote_dict = {}
for i,j in zip(Party_name,Vote_count):
    Vote_dict[i]=j

In [24]:
df2 = pd.DataFrame(list(Vote_dict.items()))

In [25]:
df2.columns=['Party','Votes']

In [26]:
df2 = df2.sort_values(['Votes'], ascending=[False])

In [27]:
total_votes = df2['Votes'].sum()

In [28]:
total_votes

645363445

In [29]:
df2['Vote_percent'] = (df2['Votes']/total_votes)*100

In [30]:
df2.head()

,Party,Votes,Vote_percent
11,BJP,235973935,36.564503
19,INC,136759064,21.191015
53,Others,45386696,7.032734
46,SP,29549381,4.578719
6,AITC,28213393,4.371706


# PART 3 - ANDHRA PRADESH

4 -> The winning party with the votes earned along with pecentage 

5 -> The seats won by parties

In [31]:
response_link_2 = requests.get(url_link[1])
response_link_2 = response_link_2.text
rs2_obj = BeautifulSoup(response_link_2,'html.parser')

In [32]:
link2_a = rs2_obj.find_all('a')
url_2 = []
for link in link2_a:
    print(link['href'])
    url_2.append(link['href'])

index.htm
index.htm
index.htm
./hi/index.htm
#
partywiseresult-S01.htm
partywiseresult-S01.htm
partywiseresult-S18.htm
partywiseresult-S18.htm
https://play.google.com/store/apps/details?id=com.eci.citizen
https://apps.apple.com/in/app/voter-helpline/id1456535004


In [33]:
url_2 = url_2[5:9]
url_2

['partywiseresult-S01.htm',
 'partywiseresult-S01.htm',
 'partywiseresult-S18.htm',
 'partywiseresult-S18.htm']

In [34]:
url_2 = url_2[1:3]

In [35]:
url_2

['partywiseresult-S01.htm', 'partywiseresult-S18.htm']

In [36]:
url_string = 'https://results.eci.gov.in/AcResultGenJune2024'
for i in range(0,len(url_2)):
    url_2[i] = url_string+'/'+url_2[i]

In [37]:
url_2

['https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S01.htm',
 'https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S18.htm']

For the link 1 -- ANDHRA PRADESH

In [38]:
response_link_andhra = requests.get(url_2[0])
response_link_andhra = response_link_andhra.text
rsa_obj = BeautifulSoup(response_link_andhra,'html.parser')

In [39]:
print(response_link_andhra)

<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1"> 
    <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'>
    <meta http-equiv='X-UA-Compatible' content='IE=7'>
    <meta http-equiv='Cache-Control' content='no-cache, no-store, must-revalidate'>
    <meta http-equiv='Pragma' content='no-cache'>
    <meta http-equiv='Expires' content='0'>
    <meta http-equiv='refresh' content='300'>
    <!-- Bootstrap CSS -->
    <link href="css/bootstrap.min.css" rel="stylesheet">
    <link href="css/custom.css" rel="stylesheet">
    <link href="css/responsive.css" rel="stylesheet">
    <link href="css/font.css" rel="stylesheet">
    <link rel='shortcut icon' href='img/icons/favicon.ico' type="image/x-icon">
    <link rel='stylesheet' href='css/leaflet.css' integrity='sha256-kLaT2GOSpHechhsozzB+flnD+zUyjE2LlfWPgU04xyI=' crossorigin='' />
    <script src='js/leaflet

In [40]:
h2_data = rsa_obj.find_all('h2')[0]
print(h2_data.span.strong.text)


# so the infromation stored on this page is about Andhra Pradesh

 Andhra Pradesh


In [41]:
xValues_andra = ['AIFB{0.05%}','BJP{2.83%}','BSP{0.60%}','CPI{0.04%}','CPI(M){0.13%}','CPI(ML)(L){0.01%}','INC{1.72%}','IUML{0.00%}','NOTA{1.09%}','RJD{0.00%}','RSP{0.00%}','SP{0.02%}','TDP{45.60%}','YSRCP{39.37%}','Other{8.53%}']
yValues_andra = [17224,953977,204060,12829,43012,1949,580613,1646,369320,119,1388,6182,15384576,13284134,2879555]

In [42]:
andra_party_name = []
for i in xValues_andra:
    x,y = i.split('{')
    andra_party_name.append(x)

In [43]:
andhra_vote = {}
for i,j in zip(andra_party_name,yValues_andra):
    andhra_vote[i]=j

In [44]:
df_andra_vote = pd.DataFrame(list(andhra_vote.items()))

In [45]:
df_andra_vote.columns=['Party','Votes']

In [46]:
total_votes_andhra = df_andra_vote['Votes'].astype(int).sum()

In [47]:
df_andra_vote['Vote_Percent'] = (df_andra_vote['Votes'].astype(int)/total_votes_andhra)*100

In [48]:
df_andra_vote

,Party,Votes,Vote_Percent
0,AIFB,17224,0.051048
1,BJP,953977,2.827387
2,BSP,204060,0.604791
3,CPI,12829,0.038022
4,CPI(M),43012,0.127479
5,CPI(ML)(L),1949,0.005776
6,INC,580613,1.720815
7,IUML,1646,0.004878
8,NOTA,369320,1.094587
9,RJD,119,0.000353


In [49]:
df_andra_vote = df_andra_vote.sort_values(['Votes'],ascending=[False])

In [50]:
df_andra_vote

,Party,Votes,Vote_Percent
12,TDP,15384576,45.596650
13,YSRCP,13284134,39.371381
14,Other,2879555,8.534396
1,BJP,953977,2.827387
6,INC,580613,1.720815
8,NOTA,369320,1.094587
2,BSP,204060,0.604791
4,CPI(M),43012,0.127479
0,AIFB,17224,0.051048
3,CPI,12829,0.038022


for the seats we could have used the table but we already have that data in the above script so using that

In [51]:
Party_seat_andhra = ['BJP','JnP','TDP','YSRCP']
seat_andhra = [8,21,135,11]

In [52]:
Seat_andhra ={}
for i,j in zip(Party_seat_andhra,seat_andhra):
    Seat_andhra[i]=j

In [53]:
Columns = ['Party','Seats']

In [54]:
df_andhra_seat = pd.DataFrame(list(Seat_andhra.items()))
df_andhra_seat.columns = Columns

In [55]:
df_andhra_seat.sort_values(['Seats'],ascending=[False])

,Party,Seats
2,TDP,135
1,JnP,21
3,YSRCP,11
0,BJP,8


In [56]:
total_seats_andhra = df_andhra_seat['Seats'].astype(int).sum()

In [57]:
df_andhra_seat['Seat_percentage'] = (df_andhra_seat['Seats']/total_seats_andhra)*100

In [58]:
df_andhra_seat.sort_values(['Seats'],ascending=[False])

,Party,Seats,Seat_percentage
2,TDP,135,77.142857
1,JnP,21,12.000000
3,YSRCP,11,6.285714
0,BJP,8,4.571429


# PART 4 - ODISHA 

6 -> Odisha wining party as well as it's seat,seat_percentage

7 -> Odisha parties vote count as well as totalvotes

In [59]:
response_link_odisha = requests.get(url_2[1])
response_link_odisha = response_link_odisha.text
rso_obj = BeautifulSoup(response_link_odisha,'html.parser')

In [60]:
table_odisha = rso_obj.find_all('table')
table_odisha

[<table class="table">
 <thead>
 <tr>
 <th style="text-align: left;">Party</th>
 <th style="text-align:right">Won</th>
 <th style="text-align:right">Leading</th>
 <th style="text-align:right">Total</th>
 </tr>
 </thead>
 <tbody>
 <tr class="tr">
 <td style="text-align:left">Bharatiya Janata Party - BJP</td>
 <td style="text-align:right">
 <a href="partywisewinresult-369S18.htm">78</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                                                     </td>
 <td style="text-align:right">78</td>
 </tr>
 <tr class="tr">
 <td style="text-align:left">Biju Janata Dal - BJD</td>
 <td style="text-align:right">
 <a href="partywisewinresult-350S18.htm">51</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                                  

In [61]:
table_odisha_header = []
for tr in rso_obj.find('tr').find_all('th'):
    print(tr.text.strip())
    table_odisha_header.append(tr.text.strip())

Party
Won
Leading
Total


In [62]:
table_odisha_data = []
for tr in rso_obj.find_all('tr')[1:]:
    for td in tr.find_all('td'):
        table_odisha_data.append(td.text.strip())

In [63]:
table_odisha_data  = np.array(table_odisha_data)

In [64]:
table_odisha_data = table_odisha_data.reshape(5,4)

In [65]:
df_odisha_seat = pd.DataFrame(table_odisha_data)

In [66]:
df_odisha_seat.columns = table_odisha_header

In [67]:
df_odisha_seat

,Party,Won,Leading,Total
0,Bharatiya Janata Party - BJP,78,0,78
1,Biju Janata Dal - BJD,51,0,51
2,Indian National Congress - INC,14,0,14
3,Communist Party of India (Marxist) - CPI(M),1,0,1
4,Independent - IND,3,0,3


In [68]:
total_seats_odisha = df_odisha_seat['Total'].astype(int).sum()

In [69]:
df_odisha_seat['Seat_Percentage'] = (df_odisha_seat['Total'].astype(int)/total_seats_odisha)*100

In [70]:
df_odisha_seat

,Party,Won,Leading,Total,Seat_Percentage
0,Bharatiya Janata Party - BJP,78,0,78,53.061224
1,Biju Janata Dal - BJD,51,0,51,34.693878
2,Indian National Congress - INC,14,0,14,9.523810
3,Communist Party of India (Marxist) - CPI(M),1,0,1,0.680272
4,Independent - IND,3,0,3,2.040816


In [71]:
print(response_link_odisha)

<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1"> 
    <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'>
    <meta http-equiv='X-UA-Compatible' content='IE=7'>
    <meta http-equiv='Cache-Control' content='no-cache, no-store, must-revalidate'>
    <meta http-equiv='Pragma' content='no-cache'>
    <meta http-equiv='Expires' content='0'>
    <meta http-equiv='refresh' content='300'>
    <!-- Bootstrap CSS -->
    <link href="css/bootstrap.min.css" rel="stylesheet">
    <link href="css/custom.css" rel="stylesheet">
    <link href="css/responsive.css" rel="stylesheet">
    <link href="css/font.css" rel="stylesheet">
    <link rel='shortcut icon' href='img/icons/favicon.ico' type="image/x-icon">
    <link rel='stylesheet' href='css/leaflet.css' integrity='sha256-kLaT2GOSpHechhsozzB+flnD+zUyjE2LlfWPgU04xyI=' crossorigin='' />
    <script src='js/leaflet

In [72]:
Party_odisha = ['AAAP{0.17%}','AIFB{0.04%}','AJSUP{0.03%}','BJD{40.22%}','BJP{40.07%}','BSP{0.38%}','CPI{0.08%}','CPI(M){0.37%}','CPI(ML)(L){0.03%}','INC{13.26%}','JMM{0.25%}','NOTA{1.02%}','SP{0.07%}','Other{4.00%}']
Vote_Odisha = [42281,10068,6749,10102454,10064827,94392,19935,93295,6881,3331319,64013,257355,18811,1003544]

In [73]:
Party_odisha_new = []
for i in Party_odisha:
    x,y = i.split('{')
    Party_odisha_new.append(x)

In [74]:
Vote_data_odisha = {}
for i,j in zip(Party_odisha_new,Vote_Odisha):
    Vote_data_odisha[i]=j

In [75]:
df_odisha_vote = pd.DataFrame(list(Vote_data_odisha.items()))

In [76]:
df_odisha_vote.columns = ['Party','Votes']
df_odisha_vote.sort_values(['Votes'],ascending=[False])

,Party,Votes
3,BJD,10102454
4,BJP,10064827
9,INC,3331319
13,Other,1003544
11,NOTA,257355
5,BSP,94392
7,CPI(M),93295
10,JMM,64013
0,AAAP,42281
6,CPI,19935


In [77]:
total_votes_odisha = df_odisha_vote['Votes'].astype(int).sum()

In [78]:
df_odisha_vote['Vote_percent'] = (df_odisha_vote['Votes'].astype(int)/total_votes_odisha)*100

In [79]:
df_odisha_vote = df_odisha_vote.sort_values(['Votes'],ascending=[False])

In [80]:
df_odisha_vote

,Party,Votes,Vote_percent
3,BJD,10102454,40.223302
4,BJP,10064827,40.073489
9,INC,3331319,13.263772
13,Other,1003544,3.995648
11,NOTA,257355,1.024669
5,BSP,94392,0.375825
7,CPI(M),93295,0.371458
10,JMM,64013,0.254870
0,AAAP,42281,0.168343
6,CPI,19935,0.079372


# PART 5 - ARUNACHAL PRADESH (AR)
 

8 -> Winning Party with no. of seat and percentage

9 -> total no. of votes in state with percentage of vote achieved by parties

In [81]:
url_link[4]

'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm'

In [82]:
response_link_4 = requests.get(url_link[4])
response_link_4 = response_link_4.text
rs4_obj = BeautifulSoup(response_link_4,'html.parser')

In [83]:
print(response_link_4)

<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1"> 
    <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'>
    <meta http-equiv='X-UA-Compatible' content='IE=7'>
    <meta http-equiv='Cache-Control' content='no-cache, no-store, must-revalidate'>
    <meta http-equiv='Pragma' content='no-cache'>
    <meta http-equiv='Expires' content='0'>
    <meta http-equiv='refresh' content='300'>
    <!-- Bootstrap CSS -->
    <link href="css/bootstrap.min.css" rel="stylesheet">
    <link href="css/custom.css" rel="stylesheet">
    <link href="css/responsive.css" rel="stylesheet">
    <link href="css/font.css" rel="stylesheet">
    <link rel='shortcut icon' href='img/icons/favicon.ico' type="image/x-icon">
    <link rel='stylesheet' href='css/leaflet.css' integrity='sha256-kLaT2GOSpHechhsozzB+flnD+zUyjE2LlfWPgU04xyI=' crossorigin='' />
    <script src='js/leaflet

In [84]:
url_3 = []

In [85]:
for i in rs4_obj.findAll('a'):
    print(i['href'])
    url_3.append(i['href'])

index.htm
index.htm
index.htm
./hi/index.htm
#
partywiseresult-S02.htm
partywiseresult-S02.htm
partywiseresult-S21.htm
partywiseresult-S21.htm
https://play.google.com/store/apps/details?id=com.eci.citizen
https://apps.apple.com/in/app/voter-helpline/id1456535004


In [86]:
url_3 = url_3[6:8]

In [87]:
url_link[4]

'https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm'

In [88]:
url_string_2 = 'https://results.eci.gov.in/AcResultGen2ndJune2024'

In [89]:
for i in range (0,len(url_3)):
    url_3[i] = url_string_2+'/'+url_3[i]

In [90]:
url_3

['https://results.eci.gov.in/AcResultGen2ndJune2024/partywiseresult-S02.htm',
 'https://results.eci.gov.in/AcResultGen2ndJune2024/partywiseresult-S21.htm']

In [91]:
response_link_ar = requests.get(url_3[0])
response_link_ar = response_link_ar.text
rsar_obj = BeautifulSoup(response_link_ar,'html.parser')

In [92]:
table_ar = rsar_obj.find_all('table')
table_ar

[<table class="table">
 <thead>
 <tr>
 <th style="text-align: left;">Party</th>
 <th style="text-align:right">Won</th>
 <th style="text-align:right">Leading</th>
 <th style="text-align:right">Total</th>
 </tr>
 </thead>
 <tbody>
 <tr class="tr">
 <td style="text-align:left">Bharatiya Janata Party - BJP</td>
 <td style="text-align:right">
 <a href="partywisewinresult-369S02.htm">46</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                                                     </td>
 <td style="text-align:right">46</td>
 </tr>
 <tr class="tr">
 <td style="text-align:left">National People's Party - NPEP</td>
 <td style="text-align:right">
 <a href="partywisewinresult-1183S02.htm">5</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                         

In [93]:
table_ar_header = []
for tr in rsar_obj.find('tr').find_all('th'):
    print(tr.text.strip())
    table_ar_header.append(tr.text.strip())

Party
Won
Leading
Total


In [94]:
table_ar_data = []
for tr in rsar_obj.find_all('tr')[1:]:
    for td in tr.find_all('td'):
        table_ar_data.append(td.text.strip())

In [95]:
table_ar_data  = np.array(table_ar_data)

In [96]:
table_ar_data.shape

(24,)

In [97]:
table_ar_data = table_ar_data.reshape(6,4)

In [98]:
df_ar_seat = pd.DataFrame(table_ar_data)
df_ar_seat.columns = table_odisha_header

In [99]:
df_ar_seat

,Party,Won,Leading,Total
0,Bharatiya Janata Party - BJP,46,0,46
1,National People's Party - NPEP,5,0,5
2,Nationalist Congress Party - NCP,3,0,3
3,People's Party of Arunachal - PPA,2,0,2
4,Indian National Congress - INC,1,0,1
5,Independent - IND,3,0,3


In [100]:
total_seats_ar = df_ar_seat['Total'].astype(int).sum()

In [101]:
df_ar_seat['Seat_Percentage'] = (df_ar_seat['Total'].astype(int)/total_seats_ar)*100
df_ar_seat

,Party,Won,Leading,Total,Seat_Percentage
0,Bharatiya Janata Party - BJP,46,0,46,76.666667
1,National People's Party - NPEP,5,0,5,8.333333
2,Nationalist Congress Party - NCP,3,0,3,5.000000
3,People's Party of Arunachal - PPA,2,0,2,3.333333
4,Indian National Congress - INC,1,0,1,1.666667
5,Independent - IND,3,0,3,5.000000


In [102]:
print(response_link_ar)

<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1"> 
    <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'>
    <meta http-equiv='X-UA-Compatible' content='IE=7'>
    <meta http-equiv='Cache-Control' content='no-cache, no-store, must-revalidate'>
    <meta http-equiv='Pragma' content='no-cache'>
    <meta http-equiv='Expires' content='0'>
    <meta http-equiv='refresh' content='300'>
    <!-- Bootstrap CSS -->
    <link href="css/bootstrap.min.css" rel="stylesheet">
    <link href="css/custom.css" rel="stylesheet">
    <link href="css/responsive.css" rel="stylesheet">
    <link href="css/font.css" rel="stylesheet">
    <link rel='shortcut icon' href='img/icons/favicon.ico' type="image/x-icon">
    <link rel='stylesheet' href='css/leaflet.css' integrity='sha256-kLaT2GOSpHechhsozzB+flnD+zUyjE2LlfWPgU04xyI=' crossorigin='' />
    <script src='js/leaflet

In [103]:
ar_party_name = ['BJP{54.57%}','INC{5.56%}','LJPRV{0.16%}','NCP{10.43%}','NOTA{0.66%}','NPEP{16.11%}','PPA{7.24%}','Other{5.26%}']
ar_vote= [332773,33877,997,63630,4010,98254,44176,32103]

In [104]:
ar_party_new = []
for i in ar_party_name:
    x,y = i.split('{')
    ar_party_new.append(x)

In [105]:
Vote_data_ar = {}
for i,j in zip(ar_party_new,ar_vote):
    Vote_data_ar[i]=j

In [106]:
df_ar_vote = pd.DataFrame(list(Vote_data_ar.items()))

In [107]:
df_ar_vote.columns = ['Party','Votes']
df_ar_vote.sort_values(['Votes'],ascending=[False])

,Party,Votes
0,BJP,332773
5,NPEP,98254
3,NCP,63630
6,PPA,44176
1,INC,33877
7,Other,32103
4,NOTA,4010
2,LJPRV,997


In [108]:
total_votes_ar = df_ar_vote['Votes'].astype(int).sum()

In [109]:
df_ar_vote['Vote_percent'] = (df_ar_vote['Votes'].astype(int)/total_votes_ar)*100

In [110]:
df_ar_vote = df_ar_vote.sort_values(['Votes'],ascending=[False])

# PART 6 - SIKKIM

10 -> Winning Party with no. of seat and percentage

11 -> total no. of votes in state with percentage of vote achieved by parties

In [111]:
response_link_s = requests.get(url_3[1])
response_link_s = response_link_s.text
rss_obj = BeautifulSoup(response_link_s,'html.parser')

In [112]:
table_s = rss_obj.find_all('table')
table_s

[<table class="table">
 <thead>
 <tr>
 <th style="text-align: left;">Party</th>
 <th style="text-align:right">Won</th>
 <th style="text-align:right">Leading</th>
 <th style="text-align:right">Total</th>
 </tr>
 </thead>
 <tbody>
 <tr class="tr">
 <td style="text-align:left">Sikkim Krantikari Morcha - SKM</td>
 <td style="text-align:right">
 <a href="partywisewinresult-1658S21.htm">31</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                                                     </td>
 <td style="text-align:right">31</td>
 </tr>
 <tr class="tr">
 <td style="text-align:left">Sikkim Democratic Front - SDF</td>
 <td style="text-align:right">
 <a href="partywisewinresult-1619S21.htm">1</a>
 </td>
 <td style="text-align:right">
                                                                                         0
                                                       

In [113]:
table_s_header = []
for tr in rss_obj.find('tr').find_all('th'):
    print(tr.text.strip())
    table_s_header.append(tr.text.strip())

Party
Won
Leading
Total


In [114]:
table_s_data = []
for tr in rss_obj.find_all('tr')[1:]:
    for td in tr.find_all('td'):
        table_s_data.append(td.text.strip())

In [115]:
table_s_data  = np.array(table_s_data)

In [116]:
table_s_data.shape

(8,)

In [117]:
table_s_data = table_s_data.reshape(2,4)

In [118]:
df_s_seat = pd.DataFrame(table_s_data)
df_s_seat.columns = table_s_header

df_s_seat

,Party,Won,Leading,Total
0,Sikkim Krantikari Morcha - SKM,31,0,31
1,Sikkim Democratic Front - SDF,1,0,1


In [119]:
total_seats_s = df_s_seat['Total'].astype(int).sum()

In [120]:
df_s_seat['Seat_Percentage'] = (df_s_seat['Total'].astype(int)/total_seats_s)*100

df_s_seat

,Party,Won,Leading,Total,Seat_Percentage
0,Sikkim Krantikari Morcha - SKM,31,0,31,96.875
1,Sikkim Democratic Front - SDF,1,0,1,3.125


In [121]:
print(response_link_s)

<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1"> 
    <meta http-equiv='Content-Type' content='text/html; charset=UTF-8'>
    <meta http-equiv='X-UA-Compatible' content='IE=7'>
    <meta http-equiv='Cache-Control' content='no-cache, no-store, must-revalidate'>
    <meta http-equiv='Pragma' content='no-cache'>
    <meta http-equiv='Expires' content='0'>
    <meta http-equiv='refresh' content='300'>
    <!-- Bootstrap CSS -->
    <link href="css/bootstrap.min.css" rel="stylesheet">
    <link href="css/custom.css" rel="stylesheet">
    <link href="css/responsive.css" rel="stylesheet">
    <link href="css/font.css" rel="stylesheet">
    <link rel='shortcut icon' href='img/icons/favicon.ico' type="image/x-icon">
    <link rel='stylesheet' href='css/leaflet.css' integrity='sha256-kLaT2GOSpHechhsozzB+flnD+zUyjE2LlfWPgU04xyI=' crossorigin='' />
    <script src='js/leaflet

In [122]:
party_s = ['BJP{5.18%}','INC{0.32%}','NOTA{0.99%}','SDF{27.37%}','SKM{58.38%}','Other{7.77%}']
vote_s= [19956,1228,3813,105503,225068,29939]

In [123]:
s_party_new = []
for i in party_s:
    x,y = i.split('{')
    s_party_new.append(x)

In [124]:
Vote_data_s = {}
for i,j in zip(s_party_new,vote_s):
    Vote_data_s[i]=j

In [125]:
df_s_vote = pd.DataFrame(list(Vote_data_s.items()))

In [126]:
df_s_vote.columns = ['Party','Votes']
df_s_vote.sort_values(['Votes'],ascending=[False])

,Party,Votes
4,SKM,225068
3,SDF,105503
5,Other,29939
0,BJP,19956
2,NOTA,3813
1,INC,1228


In [127]:
total_votes_s = df_s_vote['Votes'].astype(int).sum()

In [128]:
df_s_vote['Vote_percent'] = (df_s_vote['Votes'].astype(int)/total_votes_s)*100

In [129]:
df_s_vote = df_s_vote.sort_values(['Votes'],ascending=[False])

In [130]:
df_s_vote

,Party,Votes,Vote_percent
4,SKM,225068,58.382338
3,SDF,105503,27.367337
5,Other,29939,7.766137
0,BJP,19956,5.176560
2,NOTA,3813,0.989087
1,INC,1228,0.318542


# INSIGHTS

### 1 - TOTAL SEAT IN PARLIAMENT

In [131]:
#1 -- TOTAL NO. OF SEATS IN PARLIAMENT
print(total_parlia_seats)

543


### 2 and 3 - WINNING PARTY

winning party

In [132]:
df1['Party'][0]

'Bharatiya Janata Party - BJP'

winning seats 

In [133]:
df1['Won'][0]

'240'

winning seats percent

In [134]:
df1['Seat_Share'][0]

44.19889502762431

The total votes casted in this elecetion are :-

In [135]:
print(total_votes)

645363445


Votes acquired by BJP and it's percentage

In [136]:
df2.head(1)

,Party,Votes,Vote_percent
11,BJP,235973935,36.564503


### 4 and 5 - ANDHRA PRADESH

winning party

Total seats

In [137]:
print(total_seats_andhra)

175


seat acquired by the winning party and percentage

In [138]:
df_andhra_seat.head(1)

,Party,Seats,Seat_percentage
0,BJP,8,4.571429


total votes casted in andhra pradesh

In [139]:
print(total_votes_andhra)

33740584


votes and percentage acquired by winning party

In [140]:
df_andra_vote.head(1)

,Party,Votes,Vote_Percent
12,TDP,15384576,45.59665


### 6 and 7 - ODISHA

total seat

In [141]:
print(total_seats_odisha)

147


winning party

In [142]:
df_odisha_seat['Party'][0]

'Bharatiya Janata Party - BJP'

seats and percent acquired by winning party

In [143]:
df_odisha_seat.head(1)

,Party,Won,Leading,Total,Seat_Percentage
0,Bharatiya Janata Party - BJP,78,0,78,53.061224


total votes casted in odisha 

In [144]:
print(total_votes_odisha)

25115924


vote and percent acquired by winning party

In [145]:
df_odisha_vote.head(1)

,Party,Votes,Vote_percent
3,BJD,10102454,40.223302


It is seen that in odisha though BJD had more votes but the BJP won the election

### 8 and 9 - ARUNACHAL PRADESH

total seats 

In [146]:
total_seats_ar

60

winning party

In [147]:
df_ar_seat['Party'][0]

'Bharatiya Janata Party - BJP'

winning party seats and percent

In [148]:
df_ar_seat.head(1)

,Party,Won,Leading,Total,Seat_Percentage
0,Bharatiya Janata Party - BJP,46,0,46,76.666667


total votes in arunachal pradesh

In [149]:
total_votes_ar

609820

winning party votes and percentage

In [150]:
df_ar_vote.head(1)

,Party,Votes,Vote_percent
0,BJP,332773,54.569053


### 10 - SIKKIM

total seats

In [151]:
print(total_seats_s)

32


winning party

In [152]:
df_s_seat['Party'][0]

'Sikkim Krantikari Morcha - SKM'

seats and percentage winning party

In [153]:
df_s_seat.head(1)

,Party,Won,Leading,Total,Seat_Percentage
0,Sikkim Krantikari Morcha - SKM,31,0,31,96.875


total votes

In [154]:
total_votes_s

385507

In [155]:
df_s_vote

,Party,Votes,Vote_percent
4,SKM,225068,58.382338
3,SDF,105503,27.367337
5,Other,29939,7.766137
0,BJP,19956,5.176560
2,NOTA,3813,0.989087
1,INC,1228,0.318542


votes and percentage winning party

In [156]:
df_s_vote.head(1)

,Party,Votes,Vote_percent
4,SKM,225068,58.382338


It can be seen that in sikkim there are various coalition done inorder to form the government  